In [ ]:

from bokeh.plotting import figure
from bokeh.io import output_file, show
import pandas as pd
import numpy as np
from bokeh.plotting import ColumnDataSource
from bokeh.models.widgets import Panel
from bokeh.models.widgets import Tabs
import bokeh
from bokeh.io import curdoc
from bokeh.plotting import figure,Figure
from bokeh.layouts import row, column
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, Select
from bokeh.models.callbacks import CustomJS
from bokeh.io import curdoc
from bokeh.layouts import widgetbox
from bokeh.models import Slider
from bokeh.models import ColumnDataSource, Select,Button


In [ ]:

'''
# =============================================================================
#  running on bokeh server
#   1. open cmd of conda
#   2. get to the directory of the filename.py 
#   3. run the code:  bokeh serve --show filename.py
# =============================================================================
'''
# Perform necessary imports
from bokeh.io import curdoc
from bokeh.plotting import figure

# Create a new plot: plot
plot = figure()

# Add a line to the plot
plot.line([1,2,3,4,5],[2,5,4,6,7])

# Add the plot to the current document
curdoc().add_root(plot)



In [ ]:


# Perform the necessary imports
from bokeh.io import curdoc
from bokeh.layouts import widgetbox
from bokeh.models import Slider

# Create a slider: slider
slider = Slider(title='my slider', start=0, end=10, step=0.1, value=2)

# Create a widgetbox layout: layout
layout = widgetbox(slider)

# Add the layout to the current document
curdoc().add_root(layout)

show(layout)


In [ ]:


# Perform necessary imports
from bokeh.io import curdoc
from bokeh.layouts import widgetbox
from bokeh.models import Slider

# Create first slider: slider1
slider1 = Slider(title='slider1', start=0, end=10, step=0.1, value=2)

# Create second slider: slider2
slider2 = Slider(title='slider2', start=10, end=100, step=1, value=20)

# Add slider1 and slider2 to a widgetbox
layout = widgetbox(slider1,slider2)

# Add the layout to the current document
curdoc().add_root(layout)
show(layout)


In [ ]:


slider = Slider(title='scale', start=0, end=10, step=1, value=1)
x=np.linspace(0.3,10,300)
y=1/np.exp(x)
plot = figure()


In [ ]:

# Create ColumnDataSource: source
source = ColumnDataSource(data={'x':x,'y':y})

# Add a line to the plot
plot.line('x', 'y', source=source)

# Create a column layout: layout
layout = column(widgetbox(slider), plot)

# Add the layout to the current document
curdoc().add_root(layout)

show(layout)



In [ ]:


# Define a callback function: callback
def callback(attr, old, new):

    # Read the current value of the slider: scale
    scale = slider.value

    # Compute the updated y using np.sin(scale/x): new_y
    new_y = np.sin(scale/x)

    # Update source with the new data values
    source.data = {'x': x, 'y': new_y}

# Attach the callback to the 'value' property of slider
slider.on_change('value',callback)

# Create layout and add to current document
layout = column(widgetbox(slider), plot)
curdoc().add_root(layout)



In [ ]:


from bokeh.models.callbacks import CustomJS

x=np.linspace(0.3,10,300)
y=1/np.exp(x)

plot = Figure(plot_width=400, plot_height=400)

source = ColumnDataSource(data=dict(x=x, y=y))

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var scale = cb_obj.value
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < y.length; i++) {
        y[i] = Math.sin((scale+1)/x[i])
    }
    source.change.emit();
""")
plot.line('x', 'y', source=source)
slider = Slider(title='scale', start=0, end=10, step=1, value=0)
slider.js_on_change('value', callback)  
layout = column(slider, plot)
show(layout)



In [ ]:

df=pd.read_csv('data/literacy_birth_rate.csv')
fertility=df.fertility.tolist()
female_literacy=df.female_literacy.tolist()
population=df.population.tolist()



In [ ]:

# Perform necessary imports
from bokeh.models import ColumnDataSource, Select

# Create ColumnDataSource: source
source = ColumnDataSource(data={
    'x' : fertility,
    'y' : female_literacy
})

# Create a new plot: plot
plot = figure()

# Add circles to the plot
plot.circle('x', 'y', source=source)

# Define a callback function: update_plot
def update_plot(attr, old, new):
    # If the new Selection is 'female_literacy', update 'y' to female_literacy
    if new == 'female_literacy': 
        source.data = {
            'x' : fertility,
            'y' : female_literacy
        }
    # Else, update 'y' to population
    else:
        source.data = {
            'x' : fertility,
            'y' : population
        }

# Create a dropdown Select widget: select    
select = Select(title="distribution", options=['female_literacy', 'population'], value='female_literacy')

# Attach the update_plot callback to the 'value' property of select
select.on_change('value', update_plot)

# Create layout and add to current document
layout = row(select, plot)
curdoc().add_root(layout)



In [ ]:


# =============================================================================
#  select widget with java scripts
# =============================================================================
# Perform necessary imports
from bokeh.models import ColumnDataSource, Select,Button


source_dummy= ColumnDataSource(data={
    'x' : fertility,
    'y' : female_literacy
})

# Create ColumnDataSource: source
source_female_literacy = ColumnDataSource(data={
    'x' : fertility,
    'y' : female_literacy
})

source_population=ColumnDataSource(data = {
            'x' : fertility,
            'y' : population
        })

# Create a new plot: plot
plot = figure()

# Add circles to the plot
plot.circle('x', 'y', source=source_dummy)

update_plot = CustomJS(args={
  'source1': source_dummy, 
  'source2': source_female_literacy,
  'source3': source_population}
    , code="""
  
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var f = cb_obj.value;
  
    if (f == 'female_literacy') {
      for (var e in data1) delete data1[e];  //clears the dummy datasource
    
      data1['x'] = data2['x'];
      data1['y'] = data2['y'];
    }
  
    if (f == 'population') {
      for (var e in data1) delete data1[e];
    
      data1['x'] = data3['x']
      data1['y'] = data3['y'];
    }
       
    source1.change.emit();
""")

# Create a dropdown Select widget: select    
select = Select(title="distribution", options=['female_literacy', 'population'], value='female_literacy')

# Attach the update_plot callback to the 'value' property of select
select.js_on_change('value', update_plot)

# Create layout and add to current document
layout = row(select, plot)

show(layout)



In [ ]:


# Create two dropdown Select widgets: select1, select2
select1 = Select(title='First', options=['A', 'B'], value='A')
select2 = Select(title='Second', options=['1', '2', '3'], value='1')

# Define a callback function: callback
def callback(attr, old, new):
    # If select1 is 'A' 
    if select1.value == 'A':
        # Set select2 options to ['1', '2', '3']
        select2.options = ['1', '2', '3']

        # Set select2 value to '1'
        select2.value = '1'
    else:
        # Set select2 options to ['100', '200', '300']
        select2.options = ['100', '200', '300']

        # Set select2 value to '100'
        select2.value = '100'

# Attach the callback to the 'value' property of select1
select1.on_change('value', callback)

# Create layout and add to current document
layout = widgetbox(select1, select2)
curdoc().add_root(layout)



In [ ]:

N=200

# Create a Button with label 'Update Data'
button = Button(label='Update Data')

# Define an update callback with no arguments: update
def update():

    # Compute new y values: y
    y = np.sin(x) + np.random.random(N)

    # Update the ColumnDataSource data dictionary
    source.data = {'x': x, 'y': y}

# Add the update callback to the button
button.on_click(update)

# Create layout and add to current document
layout = column(widgetbox(button), plot)
curdoc().add_root(layout)



In [ ]:


# Import CheckboxGroup, RadioGroup, Toggle from bokeh.models
from bokeh.models import CheckboxGroup, RadioGroup, Toggle

# Add a Toggle: toggle
toggle = Toggle(button_type='success',label = 'Toggle button')

# Add a CheckboxGroup: checkbox
checkbox = CheckboxGroup(labels=['Option 1', 'Option 2', 'Option 3'])

# Add a RadioGroup: radio
radio = RadioGroup(labels=['Option 1', 'Option 2', 'Option 3'])

# Add widgetbox(toggle, checkbox, radio) to the current document
curdoc().add_root(widgetbox(toggle, checkbox, radio))


